# Ryan West Twitter API

In [133]:
import json
import requests
import pandas as pd
import urllib

# Below is the beginning of calling the API using my tokens and calling my search query to find the tweets. I will be looking at tweets from and about Nintendo (specifically Nintendo of America). The original goal was to compare the number of likes from certain posts compared to others but as you will see I was unable to do that.

In [134]:
bearer_token = pd.read_csv('TwitterBearerKey.txt', header = 0)

In [135]:
header = {'Authorization' : 'Bearer {}'.format(bearer_token['Bearer Token'].iloc[0])}

In [136]:
ep_url = 'https://api.twitter.com/2/tweets/search/recent'

In [137]:
query = urllib.parse.quote('(#Nintendo) lang:en')

In [138]:
tweet_fields = 'public_metrics,created_at,author_id,lang'

In [139]:
my_api_url = ep_url + '?query={}&max_results=100&tweet.fields={}'.format(query, tweet_fields)

In [ ]:
my_api_url

# BELOW IS WHAT I CALL MY RESPONSE 2. THIS CONTAINS THE EXPANSIONS FUNCTION. IT IS DONE OUT OF ORDER BUT I HAVE MARKED THE SECTIONS FOR EITHER RESPONSE 2 OR RESPONSE 1.

In [141]:
expansions = 'author_id'

In [142]:
url_expansions = ep_url + '?query={}&max_results=100&tweet.fields={}&expansions={}&user.fields={}'.format(query, tweet_fields, expansions, 'username')

In [143]:
response_2 = requests.request("GET", url_expansions, headers = header)

In [ ]:
response_2.json()

In [145]:
response_2_dict = json.loads(response_2.text)

In [146]:
response_2_dict.keys()

dict_keys(['data', 'includes', 'meta'])

# BELOW HERE IS MY RESPONSE 1 AKA THE ORIGINAL FUNCTION WITHOUT THE EXPANSIONS CALL.

In [147]:
response_1 = requests.request("GET", my_api_url, headers = header)

In [ ]:
response_1.text

In [ ]:
response_1.json()

In [150]:
response_1_dict = json.loads(response_1.text)

In [151]:
response_1_dict.keys()

dict_keys(['data', 'meta'])

In [152]:
my_df = pd.DataFrame(response_1_dict['data'])

In [ ]:
my_df

In [182]:
my_df.to_csv("twitter_data.csv")

# Here is the beginning of pagination up to 300 tweets.

In [184]:
next_token = url_expansions + '&next_token={}'.format(response_1_dict['meta']['next_token'])

In [185]:
next_token

'https://api.twitter.com/2/tweets/search/recent?query=%28%23Nintendo%29%20lang%3Aen&max_results=100&tweet.fields=public_metrics,created_at,author_id,lang&expansions=author_id&user.fields=username&next_token=b26v89c19zqg8o3fpdv5st2vyag83f37ojsfa39fqkyv1'

In [186]:
next_response = requests.request("GET", next_token, headers = header)

In [191]:
my_df = my_df.append(pd.DataFrame(json.loads(next_response.text)['data']), ignore_index= True)

In [ ]:
my_df

In [193]:
next_response_dict = json.loads(next_response.text)

In [194]:
next_response_dict.keys()

dict_keys(['data', 'includes', 'meta'])

In [195]:
final_token = url_expansions + '&next_token={}'.format(next_response_dict['meta']['next_token'])

In [196]:
final_token

'https://api.twitter.com/2/tweets/search/recent?query=%28%23Nintendo%29%20lang%3Aen&max_results=100&tweet.fields=public_metrics,created_at,author_id,lang&expansions=author_id&user.fields=username&next_token=b26v89c19zqg8o3fpdv5sr0v7g2lo2axq86o01pb2tn5p'

In [197]:
final_response = requests.request("GET", final_token, headers = header)

In [200]:
my_df = my_df.append(pd.DataFrame(json.loads(final_response.text)['data']), ignore_index= True)

# Below shows the dataframe calls 300 tweets without showing all of them.

In [202]:
my_df.tail(5)

,public_metrics,created_at,lang,author_id,text,id
295,"{'retweet_count': 3128, 'reply_count': 0, 'lik...",2021-10-20T18:40:44.000Z,en,3248516032,RT @Goraaz: I made a little transformation tes...,1450894772307206151
296,"{'retweet_count': 1890, 'reply_count': 0, 'lik...",2021-10-20T18:40:41.000Z,en,1171449925931950083,RT @WoodenPlank: Super Space Bros.\n\n#SuperSm...,1450894760189861890
297,"{'retweet_count': 3128, 'reply_count': 0, 'lik...",2021-10-20T18:40:23.000Z,en,738787050095316992,RT @Goraaz: I made a little transformation tes...,1450894683085881350
298,"{'retweet_count': 29, 'reply_count': 0, 'like_...",2021-10-20T18:40:11.000Z,en,846546449412313088,RT @80sThen80sNow: The Only Dog it’s Okay to H...,1450894631718330370
299,"{'retweet_count': 1890, 'reply_count': 0, 'lik...",2021-10-20T18:39:57.000Z,en,1252727790140448768,RT @WoodenPlank: Super Space Bros.\n\n#SuperSm...,1450894574176530434


In [199]:
final_response_dict = json.loads(final_response.text)

# Exported as CSV

In [203]:
my_df.to_csv(r"C:\Users\ryno2\DATAINEMERGINGMEDIA\Twitter_df.csv", encoding = 'utf-8')

In [165]:
my_df.keys()

Index(['public_metrics', 'created_at', 'lang', 'author_id', 'text', 'id'], dtype='object')

# ORIGINALLY I WAS GOING TO USE THE NUMBER OF LIKES ON POSTS TO COMPARE TO OTHER POSTS BUT I UNFORTUNATELY WAS NOT ABLE TO SEPARATE THE PUBLIC METRICS SECTION TO BE ABLE TO SORT BY THE NUMBER OF LIKES A POST HAS.

In [ ]:
my_df['public_metrics']

In [166]:
dftest = pd.DataFrame(my_df['public_metrics'])

In [ ]:
dftest

In [ ]:
liked_counts = dftest.groupby('like_count').count().sort_values('public_metrics',ascending = False)

#### above code doesnt work. I tried to use previous code to get what I wanted with the number of likes but unfortunately was unable.

# Conclusion: The data gathered shows things show as when a post was created, the id number of the user, and the actual text of the post. Using expansions, you can also retrieve the actual user name and what the person put as their actual name.

# You can also see under "public_metrics" that it lists things such as a posts retweet or like count. Originally, I wanted to use the likes of the post to compare to others and see what posts are more popular.

# As it stands now, the data gathered wouldn't be super useful unless you just specifically wanted a user's id number or perhaps compare the post times.

# This is a good start, one just needs to discover the proper code to further parse the information desired.